In [1]:
import cache
import meta
import meta_session

import nept
import numpy as np
import pandas as pd
import scipy.stats
import scipy.signal
from shapely.geometry import LineString, Point
from shapely.ops import split
import statsmodels.api as sm
import matplotlib.pyplot as plt

from utils import save_ttest_results

%matplotlib inline

In [2]:
info = meta_session.r063d6
group = "combined"

def get(key, info=None, group=None):
    if info is not None:
        return cache.load(f"ind-{info.session_id}", key)
    return cache.load(f"grp-{group}", key)

In [118]:
position_byzone = get("position_byzone", info=info)
replays_byphase = get("replays_byphase", info=info)
task_times = get("task_times", info=info)
swrs_byphase = get("swrs_byphase", info=info)
swrs_byphase_feederonly = get("swrs_byphase_feederonly", info=info)
replays_byphase_feederonly = get("replays_byphase_feederonly", info=info)
replay_n_byexperience_feederonly = get("replay_n_byexperience_feederonly", group=group)
replay_proportions_byexperience_feederonly = get("replay_proportions_byexperience_feederonly", group=group)

In [121]:
replay_n_byexperience_feederonly

{'u': {'familiar_phase12': 47,
  'first_shortcut': 1,
  'shortcut_phase3': 96,
  'familiar_phase3': 21},
 'full_shortcut': {'familiar_phase12': 47,
  'first_shortcut': 2,
  'shortcut_phase3': 98,
  'familiar_phase3': 29},
 'only_u': {'familiar_phase12': 37,
  'first_shortcut': 1,
  'shortcut_phase3': 69,
  'familiar_phase3': 15},
 'only_full_shortcut': {'familiar_phase12': 37,
  'first_shortcut': 2,
  'shortcut_phase3': 71,
  'familiar_phase3': 23},
 'both': {'familiar_phase12': 10,
  'first_shortcut': 0,
  'shortcut_phase3': 27,
  'familiar_phase3': 6}}

In [119]:
replay_n_byexperience_feederonly

{'u': {'familiar_phase12': 47,
  'first_shortcut': 1,
  'shortcut_phase3': 96,
  'familiar_phase3': 21},
 'full_shortcut': {'familiar_phase12': 47,
  'first_shortcut': 2,
  'shortcut_phase3': 98,
  'familiar_phase3': 29},
 'only_u': {'familiar_phase12': 37,
  'first_shortcut': 1,
  'shortcut_phase3': 69,
  'familiar_phase3': 15},
 'only_full_shortcut': {'familiar_phase12': 37,
  'first_shortcut': 2,
  'shortcut_phase3': 71,
  'familiar_phase3': 23},
 'both': {'familiar_phase12': 10,
  'first_shortcut': 0,
  'shortcut_phase3': 27,
  'familiar_phase3': 6}}

In [116]:
replay_proportions_byexperience_feederonly

{'u': {'familiar_phase12': 0.14826498422712933,
  'first_shortcut': 0.058823529411764705,
  'shortcut_phase3': 0.17391304347826086,
  'familiar_phase3': 0.1390728476821192},
 'full_shortcut': {'familiar_phase12': 0.14826498422712933,
  'first_shortcut': 0.11764705882352941,
  'shortcut_phase3': 0.17753623188405798,
  'familiar_phase3': 0.19205298013245034},
 'only_u': {'familiar_phase12': 0.1167192429022082,
  'first_shortcut': 0.058823529411764705,
  'shortcut_phase3': 0.125,
  'familiar_phase3': 0.09933774834437085},
 'only_full_shortcut': {'familiar_phase12': 0.1167192429022082,
  'first_shortcut': 0.11764705882352941,
  'shortcut_phase3': 0.1286231884057971,
  'familiar_phase3': 0.152317880794702},
 'both': {'familiar_phase12': 0.031545741324921134,
  'first_shortcut': 0.0,
  'shortcut_phase3': 0.04891304347826087,
  'familiar_phase3': 0.039735099337748346},
 'difference': {'familiar_phase12': 0.0,
  'first_shortcut': 0.058823529411764705,
  'shortcut_phase3': 0.00362318840579709,


In [114]:
replays_byphase.keys()

dict_keys(['u', 'full_shortcut', 'only_u', 'only_full_shortcut', 'both'])

In [100]:
replays_byphase_feederonly = get("replays_byphase_feederonly", info=info)

In [103]:
replays_byphase_feederonly["full_shortcut"]["phase3"].n_epochs

3

In [102]:
replays_byphase

{'u': {'prerecord': <nept.core.epoch.Epoch at 0x1b1b5854f48>,
  'phase1': <nept.core.epoch.Epoch at 0x1b1b5854d48>,
  'pauseA': <nept.core.epoch.Epoch at 0x1b1b5830588>,
  'phase2': <nept.core.epoch.Epoch at 0x1b1b683c808>,
  'pauseB': <nept.core.epoch.Epoch at 0x1b1b683c9c8>,
  'phase3': <nept.core.epoch.Epoch at 0x1b1b683ccc8>,
  'postrecord': <nept.core.epoch.Epoch at 0x1b1b683c388>},
 'full_shortcut': {'prerecord': <nept.core.epoch.Epoch at 0x1b1b683b988>,
  'phase1': <nept.core.epoch.Epoch at 0x1b1b683b348>,
  'pauseA': <nept.core.epoch.Epoch at 0x1b1b683b4c8>,
  'phase2': <nept.core.epoch.Epoch at 0x1b1b683bdc8>,
  'pauseB': <nept.core.epoch.Epoch at 0x1b1b683bb08>,
  'phase3': <nept.core.epoch.Epoch at 0x1b1b683b708>,
  'postrecord': <nept.core.epoch.Epoch at 0x1b1b683bf48>},
 'only_u': {'prerecord': <nept.core.epoch.Epoch at 0x1b1b683bc48>,
  'phase1': <nept.core.epoch.Epoch at 0x1b1b683bbc8>,
  'pauseA': <nept.core.epoch.Epoch at 0x1b1b683bb48>,
  'phase2': <nept.core.epoch.Ep

In [36]:
replays_centers = replays.centers
replays_centers

array([7111.510146, 7179.519146, 7215.224896, 7305.988146, 7555.133146,
       7615.182646, 7875.802146, 7990.097646, 8242.951396, 8256.684896,
       8263.660146, 8356.501646, 8686.449896, 8791.817646, 8818.667396,
       9039.863396, 9083.460896, 9177.106146, 9216.514396, 9557.821896])

In [23]:
position_byzone["u_feeder1"]

In [94]:
replays_infeeders = {trajectory: {phase: [] for phase in meta.run_times} for trajectory in meta.trajectories}
for trajectory in meta.trajectories:
    for phase in meta.run_times:
        starts = []
        stops = []
        for replay in replays_byphase[trajectory][phase]:
            for feeder in ["u_feeder1", "u_feeder2", "full_shortcut_feeder1", "full_shortcut_feeder2"]:
                feeder_time = position_byzone[feeder].time
                if replay.starts not in starts:
                    if np.allclose(replay.centers, feeder_time[nept.find_nearest_idx(position_byzone[feeder].time, replay.centers)]):
                        starts.append(replay.start)
                        stops.append(replay.stop)
        if len(starts) > 0:
            replays_infeeders[trajectory][phase] = nept.Epoch(starts, stops)
        else:
            replays_infeeders[trajectory][phase] = nept.Epoch([], [])

In [97]:
replays_infeeders["u"]["phase1"].starts

array([], dtype=float64)

In [90]:
nept.Epoch([], [])